ResNet Ensemble Working File

In [9]:
# === Data / Dataloaders for Ensemble Notebook ===

import os, csv
from pathlib import Path
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# ---- Paths ----
CSV_DIR    = Path(r"C:\Users\Andre\Documents\Machine Learning Project\processed_csvs")
IMAGE_ROOT = CSV_DIR  # images live under this root

# ---- Label mapping (must match training) ----
LABEL_TO_IDX = {"clear": 0, "obstructed": 1}

# ---- Normalization (ImageNet) ----
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

def get_transforms():
    """Baseline transforms (no RandAugment) – same as your baseline models."""
    train_tf = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ])

    eval_tf = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])
    return train_tf, eval_tf

train_tf, eval_tf = get_transforms()

class CSVDataset(Dataset):
    def __init__(self, csv_path, transform, image_root=None):
        self.rows = []
        self.transform = transform
        self.image_root = image_root

        with open(csv_path, "r", newline="") as f:
            r = csv.DictReader(f)
            assert {"filename","label"}.issubset(r.fieldnames), f"Missing headers in {csv_path}"
            for row in r:
                fp = row["filename"].strip()
                if image_root is not None and not os.path.isabs(fp):
                    fp = str(Path(image_root) / fp)
                lab = row["label"].strip().lower()
                assert lab in LABEL_TO_IDX, f"Unknown label {lab} in {csv_path}"
                self.rows.append((fp, LABEL_TO_IDX[lab]))

    def __len__(self):
        return len(self.rows)

    def __getitem__(self, idx):
        img_path, y = self.rows[idx]
        with Image.open(img_path) as im:
            im = im.convert("RGB")
        x = self.transform(im)
        return x, torch.tensor(y, dtype=torch.long)

def make_loaders(batch_size=64, num_workers=0, pin_memory=False):
    """Create train/val/test loaders from train.csv, val.csv, test.csv."""
    train_csv = CSV_DIR / "train.csv"
    val_csv   = CSV_DIR / "val.csv"
    test_csv  = CSV_DIR / "test.csv"

    train_ds = CSVDataset(train_csv, transform=train_tf, image_root=IMAGE_ROOT)
    val_ds   = CSVDataset(val_csv,   transform=eval_tf,  image_root=IMAGE_ROOT)
    test_ds  = CSVDataset(test_csv,  transform=eval_tf,  image_root=IMAGE_ROOT)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,
                              num_workers=num_workers, pin_memory=pin_memory)
    val_loader   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False,
                              num_workers=num_workers, pin_memory=pin_memory)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False,
                              num_workers=num_workers, pin_memory=pin_memory)

    return train_loader, val_loader, test_loader, train_ds, val_ds, test_ds

# Instantiate loaders (we only really need test_loader here, but this is fine)
device = torch.device("cpu")  # keep everything CPU for ensemble
train_loader, val_loader, test_loader, train_ds, val_ds, test_ds = make_loaders(
    batch_size=64,
    num_workers=0,
    pin_memory=False
)

print("Label mapping:", LABEL_TO_IDX)
print("Train size:", len(train_ds), "Val size:", len(val_ds), "Test size:", len(test_ds))


Label mapping: {'clear': 0, 'obstructed': 1}
Train size: 14000 Val size: 3000 Test size: 3000


In [10]:
# === Load Baseline ResNet18 & ResNet50 (FP32) ===

import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cpu")
NUM_CLASSES = 2

# -------------------------------
# Builder: Baseline ResNet18
# -------------------------------
def build_baseline_r18(weights_path: str):
    model = models.resnet18(weights=None)
    in_feats = model.fc.in_features
    model.fc = nn.Linear(in_feats, NUM_CLASSES)

    state = torch.load(weights_path, map_location=device)
    model.load_state_dict(state)
    model.to(device).eval()
    return model

# -------------------------------
# Builder: Baseline ResNet50
# -------------------------------
def build_baseline_r50(weights_path: str):
    model = models.resnet50(weights=None)
    in_feats = model.fc.in_features
    model.fc = nn.Linear(in_feats, NUM_CLASSES)

    state = torch.load(weights_path, map_location=device)
    model.load_state_dict(state)
    model.to(device).eval()
    return model

# -------------------------------
# Load both baselines
# -------------------------------
R18_BASE = "resnet18_clear_obstructed_best.pt"
R50_BASE = "resnet50_clear_obstructed_best.pt"

baseline_r18 = build_baseline_r18(R18_BASE)
baseline_r50 = build_baseline_r50(R50_BASE)

print("Loaded baseline ResNet18 & ResNet50 successfully.")


C:\Users\Andre\AppData\Local\Temp\ipykernel_33680\3003938082.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(weights_path, map_location=device)


Loaded baseline ResNet18 & ResNet50 successfully.


C:\Users\Andre\AppData\Local\Temp\ipykernel_33680\3003938082.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(weights_path, map_location=device)


In [11]:
# === Build & Load FX-Quantized ResNet18 and ResNet50 (INT8) ===

import torch
import torch.nn as nn
from torchvision import models

from torch.ao.quantization import get_default_qconfig, QConfigMapping
try:
    from torch.ao.quantization.quantize_fx import prepare_fx, convert_fx
except ImportError:
    from torch.quantization.quantize_fx import prepare_fx, convert_fx

device = torch.device("cpu")
NUM_CLASSES = 2

R18_QUANT_SD = "resnet18_clear_obstructed_quant_fx.pt"   # state_dict you already saved
R50_QUANT_SD = "resnet50_clear_obstructed_quant_fx.pt"   # state_dict you already saved


def build_resnet18_quant_shell() -> nn.Module:
    """Rebuild quantized ResNet18 graph (no calibration), ready for loading quant SD."""
    # 1) Float model with 2-class head
    m = models.resnet18(weights=None)
    in_feats = m.fc.in_features
    m.fc = nn.Linear(in_feats, NUM_CLASSES)
    m.eval()

    # 2) QConfig & FX prepare/convert
    qconfig = get_default_qconfig("fbgemm")
    qconfig_mapping = QConfigMapping().set_global(qconfig)
    example_inputs = torch.randn(1, 3, 224, 224)

    prepared = prepare_fx(m, qconfig_mapping, example_inputs)
    quantized = convert_fx(prepared)
    quantized.eval()
    return quantized


def build_resnet50_quant_shell() -> nn.Module:
    """Rebuild quantized ResNet50 graph (no calibration), ready for loading quant SD."""
    m = models.resnet50(weights=None)
    in_feats = m.fc.in_features
    m.fc = nn.Linear(in_feats, NUM_CLASSES)
    m.eval()

    qconfig = get_default_qconfig("fbgemm")
    qconfig_mapping = QConfigMapping().set_global(qconfig)
    example_inputs = torch.randn(1, 3, 224, 224)

    prepared = prepare_fx(m, qconfig_mapping, example_inputs)
    quantized = convert_fx(prepared)
    quantized.eval()
    return quantized


# Build FX-quantized shells
quant_r18 = build_resnet18_quant_shell()
quant_r50 = build_resnet50_quant_shell()

# Load the calibrated + converted weights into those shells
quant_r18.load_state_dict(torch.load(R18_QUANT_SD, map_location=device))
quant_r50.load_state_dict(torch.load(R50_QUANT_SD, map_location=device))

quant_r18.to(device).eval()
quant_r50.to(device).eval()

print("Loaded FX-quantized ResNet18 & ResNet50 (via FX shell + state_dict) successfully.")


x:\miniconda3\envs\cv-efficiency\Lib\site-packages\torch\ao\quantization\observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
x:\miniconda3\envs\cv-efficiency\Lib\site-packages\torch\ao\quantization\observer.py:1315: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(
x:\miniconda3\envs\cv-efficiency\Lib\site-packages\torch\ao\quantization\observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
x:\miniconda3\envs\cv-efficiency\Lib\site-packages\torch\ao\quantization\observer.py:1315: UserWarning: must run observer before calling calculate_qparams.                                    Returning d

Loaded FX-quantized ResNet18 & ResNet50 (via FX shell + state_dict) successfully.


C:\Users\Andre\AppData\Local\Temp\ipykernel_33680\779848892.py:61: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  quant_r18.load_state_dict(torch.load(R18_QUANT_SD, map_locat

In [12]:
# === Ensemble Evaluation: ResNet18 + ResNet50 (Baseline / Quantized / Both) ===

import torch

device = torch.device("cpu")   # keep everything on CPU for consistency with quant models


def eval_ensemble(models, loader, name="Ensemble"):
    """
    Evaluate an ensemble of models on the test set.
    - models: list of nn.Modules
    - loader: DataLoader (test_loader)
    Returns: (acc, cm, metrics_dict)
    """
    for m in models:
        m.to(device)
        m.eval()

    cm = torch.zeros(2, 2, dtype=torch.long)  # [[TN, FP],[FN, TP]]
    total, correct = 0, 0

    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)

            # Sum logits from all models (mean and sum give same argmax)
            logits_sum = None
            for m in models:
                out = m(xb)
                logits_sum = out if logits_sum is None else (logits_sum + out)

            logits = logits_sum / len(models)
            pred = logits.argmax(1)

            # update confusion matrix
            for t, p in zip(yb.view(-1), pred.view(-1)):
                cm[t.long(), p.long()] += 1

            correct += (pred == yb).sum().item()
            total   += yb.size(0)

    acc = correct / total

    TN, FP = cm[0,0].item(), cm[0,1].item()
    FN, TP = cm[1,0].item(), cm[1,1].item()

    def safe_div(a, b): 
        return a / b if b > 0 else 0.0

    # Same "clear"/"obstructed" convention as everywhere else
    prec_clear = safe_div(TN, TN + FN)
    rec_clear  = safe_div(TN, TN + FP)
    f1_clear   = safe_div(
        2 * prec_clear * rec_clear,
        prec_clear + rec_clear
    ) if (prec_clear + rec_clear) > 0 else 0.0

    prec_obst = safe_div(TP, TP + FP)
    rec_obst  = safe_div(TP, TP + FN)
    f1_obst   = safe_div(
        2 * prec_obst * rec_obst,
        prec_obst + rec_obst
    ) if (prec_obst + rec_obst) > 0 else 0.0

    print(f"\n=== {name} ===")
    print("Test accuracy:", f"{acc:.4f}")
    print("Confusion matrix (rows=true, cols=pred):")
    print(cm.tolist())
    print("Per-class metrics:")
    print(f"  clear (0):       precision {prec_clear:.4f}  recall {rec_clear:.4f}  f1 {f1_clear:.4f}")
    print(f"  obstructed (1):  precision {prec_obst:.4f}  recall {rec_obst:.4f}  f1 {f1_obst:.4f}")

    return acc, cm, {
        "clear":      (prec_clear, rec_clear, f1_clear),
        "obstructed": (prec_obst, rec_obst, f1_obst),
    }


# 1) Baseline ensemble: ResNet18 + ResNet50 (FP32)
ens_base_acc, ens_base_cm, ens_base_metrics = eval_ensemble(
    [baseline_r18, baseline_r50],
    test_loader,
    name="Baseline Ensemble (ResNet18 + ResNet50, FP32)"
)

# 2) Quantized ensemble: ResNet18 + ResNet50 (FX INT8)
ens_quant_acc, ens_quant_cm, ens_quant_metrics = eval_ensemble(
    [quant_r18, quant_r50],
    test_loader,
    name="Quantized Ensemble (ResNet18 + ResNet50, FX INT8)"
)

# 3) Full ensemble: all four models together (optional but interesting)
ens_all_acc, ens_all_cm, ens_all_metrics = eval_ensemble(
    [baseline_r18, baseline_r50, quant_r18, quant_r50],
    test_loader,
    name="Full Ensemble (Baseline + Quantized ResNets)"
)



=== Baseline Ensemble (ResNet18 + ResNet50, FP32) ===
Test accuracy: 0.9253
Confusion matrix (rows=true, cols=pred):
[[1600, 129], [95, 1176]]
Per-class metrics:
  clear (0):       precision 0.9440  recall 0.9254  f1 0.9346
  obstructed (1):  precision 0.9011  recall 0.9253  f1 0.9130

=== Quantized Ensemble (ResNet18 + ResNet50, FX INT8) ===
Test accuracy: 0.9270
Confusion matrix (rows=true, cols=pred):
[[1606, 123], [96, 1175]]
Per-class metrics:
  clear (0):       precision 0.9436  recall 0.9289  f1 0.9362
  obstructed (1):  precision 0.9052  recall 0.9245  f1 0.9148

=== Full Ensemble (Baseline + Quantized ResNets) ===
Test accuracy: 0.9257
Confusion matrix (rows=true, cols=pred):
[[1602, 127], [96, 1175]]
Per-class metrics:
  clear (0):       precision 0.9435  recall 0.9265  f1 0.9349
  obstructed (1):  precision 0.9025  recall 0.9245  f1 0.9133


In [13]:
# === Latency & Throughput Benchmark for ResNet Ensembles (CPU) ===

import time
import numpy as np
import torch

device = torch.device("cpu")  # keep CPU for fair comparison


@torch.no_grad()
def benchmark_ensemble_cpu(models, loader, warmup: int = 20, iters: int = 200, name: str = "Ensemble"):
    """
    Benchmark an ensemble of models on CPU:
      - single-image latency
      - batch latency & throughput

    Assumes:
      - models: list of nn.Module
      - loader: DataLoader (we'll grab a single batch from it)
    """
    # Move all models to CPU + eval
    models = [m.to(device).eval() for m in models]

    # Grab one batch from the loader
    xbB, _ = next(iter(loader))
    xbB = xbB.to(device)      # [B, 3, 224, 224]
    xb1 = xbB[:1]             # [1, 3, 224, 224]

    times_single, times_batch = [], []

    # Warmup
    for _ in range(warmup):
        # single image
        logits_sum = None
        for m in models:
            out = m(xb1)
            logits_sum = out if logits_sum is None else (logits_sum + out)

        # batch
        logits_sum = None
        for m in models:
            out = m(xbB)
            logits_sum = out if logits_sum is None else (logits_sum + out)

    # Single-image timing
    for _ in range(iters):
        t0 = time.perf_counter()
        logits_sum = None
        for m in models:
            out = m(xb1)
            logits_sum = out if logits_sum is None else (logits_sum + out)
        times_single.append(time.perf_counter() - t0)

    # Batch timing
    for _ in range(iters):
        t0 = time.perf_counter()
        logits_sum = None
        for m in models:
            out = m(xbB)
            logits_sum = out if logits_sum is None else (logits_sum + out)
        times_batch.append(time.perf_counter() - t0)

    def stats(ts):
        ts = np.array(ts) * 1000.0  # sec → ms
        return ts.mean(), np.percentile(ts, 95)

    m1, p951 = stats(times_single)
    mB, p95B = stats(times_batch)
    bsz = xbB.size(0)

    fps_single = 1000.0 / m1
    fps_batch  = (bsz * 1000.0) / mB

    print(f"\n=== {name} (CPU) ===")
    print(f"Single image:  mean {m1:.2f} ms  p95 {p951:.2f} ms  FPS ~{fps_single:.1f}")
    print(f"Batch ({bsz}): mean {mB:.2f} ms  p95 {p95B:.2f} ms  Throughput ~{fps_batch:.1f} img/s")

    return {
        "single_mean_ms": m1,
        "single_p95_ms": p951,
        "single_fps": fps_single,
        "batch_mean_ms": mB,
        "batch_p95_ms": p95B,
        "batch_fps": fps_batch,
        "batch_size": bsz,
    }


# ---- Run benchmarks for the three ensembles ----

ens_base_stats = benchmark_ensemble_cpu(
    [baseline_r18, baseline_r50],
    test_loader,
    name="Baseline Ensemble (ResNet18 + ResNet50, FP32)"
)

ens_quant_stats = benchmark_ensemble_cpu(
    [quant_r18, quant_r50],
    test_loader,
    name="Quantized Ensemble (ResNet18 + ResNet50, FX INT8)"
)

ens_all_stats = benchmark_ensemble_cpu(
    [baseline_r18, baseline_r50, quant_r18, quant_r50],
    test_loader,
    name="Full Ensemble (Baseline + Quantized ResNets)"
)



=== Baseline Ensemble (ResNet18 + ResNet50, FP32) (CPU) ===
Single image:  mean 27.31 ms  p95 28.89 ms  FPS ~36.6
Batch (64): mean 1360.57 ms  p95 1389.96 ms  Throughput ~47.0 img/s

=== Quantized Ensemble (ResNet18 + ResNet50, FX INT8) (CPU) ===
Single image:  mean 12.39 ms  p95 13.39 ms  FPS ~80.7
Batch (64): mean 232.23 ms  p95 249.53 ms  Throughput ~275.6 img/s

=== Full Ensemble (Baseline + Quantized ResNets) (CPU) ===
Single image:  mean 40.75 ms  p95 42.50 ms  FPS ~24.5
Batch (64): mean 1625.86 ms  p95 1730.03 ms  Throughput ~39.4 img/s


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ======================================================
# Hard-coded metrics from your logs (NO recomputation)
# ======================================================

labels = [
    "R18 FP32",
    "R18 INT8",
    "R50 FP32",
    "R50 INT8",
    "Ens FP32",
    "Ens INT8",
    "Ens Full",
]

# ---- Accuracy ----
acc = np.array([
    0.9147,  # R18 baseline (CPU summary)
    0.9150,  # R18 quant
    0.9180,  # R50 baseline
    0.9177,  # R50 quant
    0.9253,  # Ensemble baseline
    0.9270,  # Ensemble quant
    0.9257,  # Ensemble full
])

# ---- Per-class recalls (from your text) ----
# Clear (class 0) recall
rec_clear = np.array([
    0.9190,  # R18 base
    0.9213,  # R18 quant
    0.9161,  # R50 base
    0.9202,  # R50 quant
    0.9254,  # Ens base
    0.9289,  # Ens quant
    0.9265,  # Ens full
])

# Obstructed (class 1) recall
rec_obst = np.array([
    0.9087,  # R18 base
    0.9064,  # R18 quant
    0.9205,  # R50 base
    0.9142,  # R50 quant
    0.9253,  # Ens base
    0.9245,  # Ens quant
    0.9245,  # Ens full
])

# Obstructed F1
f1_obst = np.array([
    0.9002,  # R18 base
    0.9004,  # R18 quant
    0.9049,  # R50 base
    0.9039,  # R50 quant
    0.9130,  # Ens base
    0.9148,  # Ens quant
    0.9133,  # Ens full
])

# ---- Throughput (CPU) ----
# Single-image FPS
fps_single = np.array([
    132.0,  # R18 base
    289.1,  # R18 quant
    50.1,   # R50 base
    121.5,  # R50 quant
    36.6,   # Ens base
    80.7,   # Ens quant
    24.5,   # Ens full
])

# Batch throughput (img/s)
throughput_batch = np.array([
    213.4,  # R18 base
    1069.8, # R18 quant
    58.8,   # R50 base
    374.8,  # R50 quant
    47.0,   # Ens base
    275.6,  # Ens quant
    39.4,   # Ens full
])

# ======================================================
# Plot – 2×2 grid
# ======================================================
plt.style.use("default")
fig, axes = plt.subplots(2, 2, figsize=(14, 8))
fig.suptitle("ResNet18 / ResNet50 – Singles vs Ensembles (Metrics & CPU Throughput)",
             fontsize=16, weight="bold")

x = np.arange(len(labels))
bar_width = 0.6

# Color mapping (just for visual grouping)
colors = {
    "R18 FP32":    "#1f77b4",
    "R18 INT8":    "#9467bd",
    "R50 FP32":    "#1f77b4",
    "R50 INT8":    "#9467bd",
    "Ens FP32":    "#2ca02c",
    "Ens INT8":    "#d62728",
    "Ens Full":    "#7f7f7f",
}
bar_colors = [colors[l] for l in labels]

# ---------- (a) Test accuracy ----------
ax = axes[0, 0]
ax.bar(x, acc, width=bar_width, color=bar_colors)
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=20, ha="right")
ax.set_ylabel("Accuracy")
ax.set_title("Test Accuracy")
ax.set_ylim(acc.min() - 0.01, acc.max() + 0.01)
for i, v in enumerate(acc):
    ax.text(i, v + 0.001, f"{v:.3f}", ha="center", va="bottom", fontsize=8)
ax.grid(axis="y", linestyle="--", alpha=0.3)

# ---------- (b) Obstructed recall ----------
ax = axes[0, 1]
ax.bar(x, rec_obst, width=bar_width, color=bar_colors)
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=20, ha="right")
ax.set_ylabel("Recall (obstructed)")
ax.set_title("Per-class Recall – Obstructed (Class 1)")
ax.set_ylim(rec_obst.min() - 0.01, rec_obst.max() + 0.01)
for i, v in enumerate(rec_obst):
    ax.text(i, v + 0.001, f"{v:.3f}", ha="center", va="bottom", fontsize=8)
ax.grid(axis="y", linestyle="--", alpha=0.3)

# ---------- (c) Single-image FPS (CPU) ----------
ax = axes[1, 0]
ax.bar(x, fps_single, width=bar_width, color=bar_colors)
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=20, ha="right")
ax.set_ylabel("FPS (single image, CPU)")
ax.set_title("Single-image Inference Speed (CPU)")
ax.set_ylim(0, fps_single.max() * 1.15)
for i, v in enumerate(fps_single):
    ax.text(i, v + fps_single.max()*0.02, f"{v:.1f}", ha="center", va="bottom", fontsize=8)
ax.grid(axis="y", linestyle="--", alpha=0.3)

# ---------- (d) Batch throughput (CPU) ----------
ax = axes[1, 1]
ax.bar(x, throughput_batch, width=bar_width, color=bar_colors)
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=20, ha="right")
ax.set_ylabel("images / second (batch=64, CPU)")
ax.set_title("Batch Throughput (CPU, 64 images)")
ax.set_ylim(0, throughput_batch.max() * 1.15)
for i, v in enumerate(throughput_batch):
    ax.text(i, v + throughput_batch.max()*0.02, f"{v:.1f}", ha="center", va="bottom", fontsize=8)
ax.grid(axis="y", linestyle="--", alpha=0.3)

plt.tight_layout(rect=[0, 0, 1, 0.94])
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# -----------------------------
# Hard-coded metrics
# -----------------------------
# Baseline: single ResNet50 FP32 (CPU)
acc_r50_cpu        = 0.9180
rec_obst_r50_cpu   = 0.9205
fps_single_r50_cpu = 50.1
thr_batch_r50_cpu  = 58.8

# Hero: Quantized ResNet Ensemble (R18 + R50, FX INT8, CPU)
acc_ens_q        = 0.9270
rec_obst_ens_q   = 0.9245
fps_single_ens_q = 80.7
thr_batch_ens_q  = 275.6

metrics = [
    "Accuracy",
    "Obstructed Recall",
    "Single FPS (CPU)",
    "Batch Throughput (CPU)",
]

baseline_vals = np.array([
    acc_r50_cpu,
    rec_obst_r50_cpu,
    fps_single_r50_cpu,
    thr_batch_r50_cpu,
])

ensemble_vals = np.array([
    acc_ens_q,
    rec_obst_ens_q,
    fps_single_ens_q,
    thr_batch_ens_q,
])

ratios = ensemble_vals / baseline_vals   # normalized to R50 FP32 = 1.0
pct_deltas = (ratios - 1.0) * 100.0      # percentage change

plt.style.use("default")
fig, ax = plt.subplots(figsize=(8, 5))

y = np.arange(len(metrics))

ax.barh(y, ratios, color="purple", alpha=0.7)
ax.axvline(1.0, color="gray", linestyle="--", linewidth=1.0)

ax.set_yticks(y)
ax.set_yticklabels(metrics)
ax.set_xlabel("Normalized to ResNet50 FP32 (CPU)  [Baseline = 1.0]")
ax.set_title(
    "Quantized ResNet Ensemble (R18 + R50, FX INT8, CPU)\n"
    "vs Single ResNet50 FP32 (CPU)"
)

# Annotate each bar with ratio + % change
for i, (r, d) in enumerate(zip(ratios, pct_deltas)):
    ax.text(
        r + 0.02, i,
        f"{r:.2f}  ({d:+.1f}%)",
        va="center",
        fontsize=9,
    )

ax.set_xlim(0.9, max(1.1, ratios.max() + 1.0))
ax.grid(axis="x", linestyle="--", alpha=0.3)

plt.tight_layout()
plt.show()
